In [1]:
# ================================================================
#  Unified EIS Training + Inference + Dynamic RUL  (v8 – single file)
# ================================================================
#  – helper code lives in eis_helpers.py  (sections 5–14 from v7)
# ================================================================

from __future__ import annotations
import sys, argparse, json, math, random, re
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Any, Optional

# ---------- unchanged helpers (v7 sections 5–14) -----------------
from eis_helpers import (          # <- your v7 code lives here
    CANON_FREQ, set_seed, to_jsonable,
    parse_eis_metadata, parse_cap_metadata,
    load_capacity_info, build_cpp_map, get_cpp,
    featurize_any, build_dataset, train_models,
    load_bundle, plot_projection
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor

# =========================
# 1. CONFIGURATION
# =========================
@dataclass
class Config:
    # --- your local folders & test file --------------------------
    EIS_DIR: Path = Path(r"C:\Users\tmgon\OneDrive - Edith Cowan University \00 - Megallan Power\NMC Batteries Warwick Station\NMC\DIB_Data\.matfiles\EIS_Test")
    CAP_DIR: Path = Path(r"C:\Users\tmgon\OneDrive - Edith Cowan University \00 - Megallan Power\NMC Batteries Warwick Station\NMC\DIB_Data\.matfiles\Capacity_Check")
    MODEL_DIR: Path = Path("models_eis_phase2_phys")
    EIS_TEST_FILE: Path = Path(r"C:\Users\tmgon\OneDrive - Edith Cowan University \00 - Megallan Power\NMC Batteries Warwick Station\NMC\TestFile\Mazda-Battery-Cell1.xlsx")

    # ---- other hyper-params (identical to v7 except FEATURE_VERSION) ----
    F_MIN: float = 1e-2; F_MAX: float = 1e4; N_FREQ: int = 60
    TEST_FRAC: float = 0.2; RANDOM_STATE: int = 42
    USE_PCA_SOC: bool = True; USE_PCA_SOH: bool = False
    PCA_SOC_COMPONENTS: int = 25; PCA_SOH_COMPONENTS: int = 30
    INCLUDE_RAW_RE_IM: bool = True; INCLUDE_BASICS: bool = True
    INCLUDE_F_FEATS: bool = True;  INCLUDE_PHYSICAL: bool = True
    INCLUDE_DRT: bool = True;      INCLUDE_BAND_STATS: bool = True
    INCLUDE_DIFF_SLOPES: bool = True
    DRT_POINTS: int = 60; DRT_TAU_MIN: float = 1e-4; DRT_TAU_MAX: float = 1e4; DRT_LAMBDA: float = 1e-2
    REFINE_SOH_WITH_CAPACITY: bool = True
    MAX_GPR_TRAIN_SAMPLES: int = 3500
    INCLUDE_NORMALIZED_SHAPE_MODEL: bool = True; ENSEMBLE_SOH: bool = True
    NORMALIZE_SHAPE_BY_HF_RE: bool = True
    DECISION_SOH_PERCENT: float = 50.0; ILLUSTRATIVE_MIN_SOH: float = 40.0
    CPP_ROLLING_WINDOW: int = 5; CPP_MIN_POINTS: int = 6; CPP_FALLBACK: float = 20.0
    TEST_TEMPERATURE_OVERRIDE: Optional[float] = 25.0; FORCE_RETRAIN: bool = False
    SAVE_FEATURE_TABLE: bool = True; VERBOSE: bool = True; FEATURE_VERSION: int = 8
    MAHAL_THRESHOLD: float = 10.0; GP_ARD_NORM_THRESHOLD: float = 6.0
    PLOT_EXPONENT: float = 1.25

cfg = Config(); cfg.MODEL_DIR.mkdir(parents=True, exist_ok=True)
set_seed(cfg.RANDOM_STATE)

# =========================
# 2. INFERENCE for single file
# =========================
def predict_file(fp: Path, bundle, cpp_map, global_cpp):
    vec, norm_vec, meta = featurize_any(fp, bundle)

    # ----- SoC ----------------------------------------------------
    soc_scaler, soc_pca, soc_model = bundle["soc_scaler"], bundle.get("soc_pca"), bundle["soc_model"]
    Xs = soc_scaler.transform(vec.reshape(1, -1))
    Xs = soc_pca.transform(Xs) if soc_pca is not None else Xs
    probs = soc_model.predict_proba(Xs)[0]
    soc = int(soc_model.classes_[probs.argmax()])

    # ----- SoH ----------------------------------------------------
    soh_scaler, soh_pca, soh_mdl = bundle["soh_scaler"], bundle.get("soh_pca"), bundle["soh_model"]
    Xr = soh_scaler.transform(vec.reshape(1, -1))
    Xr = soh_pca.transform(Xr) if soh_pca is not None else Xr
    if isinstance(soh_mdl, GaussianProcessRegressor):
        mu, sigma = soh_mdl.predict(Xr, return_std=True)
        soh_mean, soh_std = float(mu[0]), float(sigma[0])
    else:
        soh_mean = float(soh_mdl.predict(Xr)[0]); soh_std = float(bundle["metrics"].get("soh_rmse_selected", 5.0))
    soh_std = min(soh_std, 5.0)           # ← clamp ±5 pp

    # optional shape GP ensemble
    if cfg.ENSEMBLE_SOH and bundle.get("shape_model") and norm_vec is not None:
        sh_scl, sh_pca, sh_mdl = bundle["shape_scaler"], bundle.get("shape_pca"), bundle["shape_model"]
        Xn = sh_scl.transform(norm_vec.reshape(1, -1))
        Xn = sh_pca.transform(Xn) if sh_pca is not None else Xn
        smu = float(sh_mdl.predict(Xn)[0])
        soh_mean = 0.5*(soh_mean + smu)

    cpp = get_cpp(meta, cpp_map, global_cpp)
    cyc_target = max((soh_mean - cfg.DECISION_SOH_PERCENT)*cpp, 0)
    cyc_lower  = max((soh_mean - cfg.ILLUSTRATIVE_MIN_SOH)*cpp, 0)

    return {
        "file": str(fp), "parsed_metadata": meta,
        "predicted_SoC": soc,
        "SoC_probabilities": {int(c): float(p) for c, p in zip(soc_model.classes_, probs)},
        "predicted_SoH_percent": soh_mean, "SoH_std_estimate": soh_std,
        "cycles_per_percent_used": cpp, "cycles_to_target": cyc_target,
        "cycles_to_lower": cyc_lower, "decision_threshold_percent": cfg.DECISION_SOH_PERCENT,
        "lower_threshold_percent": cfg.ILLUSTRATIVE_MIN_SOH,
        "feature_version": bundle["feature_version"], "soh_model_chosen": bundle.get("soh_model_name","raw")
    }

# =========================
# 3. MAIN
# =========================
def main(argv=None):
    # CLI parsing resilient to Jupyter '-f …'
    p = argparse.ArgumentParser(add_help=False)
    p.add_argument("--test", dest="test_file")
    args, _ = p.parse_known_args([] if argv is None else argv)
    if args.test_file: cfg.EIS_TEST_FILE = Path(args.test_file)

    if cfg.VERBOSE: print("Configuration:\n", json.dumps(to_jsonable(asdict(cfg)), indent=2))
    assert cfg.EIS_DIR.exists(), "EIS_DIR missing"; assert cfg.CAP_DIR.exists(), "CAP_DIR missing"

    cap_df = load_capacity_info(cfg.CAP_DIR)
    cpp_map, global_cpp = build_cpp_map(cap_df) if not cap_df.empty else ({}, cfg.CPP_FALLBACK)

    bundle_path = cfg.MODEL_DIR / "eis_soc_soh_phys_models.joblib"
    bundle = load_bundle() if bundle_path.exists() and not cfg.FORCE_RETRAIN else \
             train_models(*build_dataset(cfg.EIS_DIR, cap_df))

    fp = cfg.EIS_TEST_FILE
    if not fp.exists(): raise FileNotFoundError(fp)
    res = predict_file(fp, bundle, cpp_map, global_cpp)

    plot_projection(fp.stem, res["predicted_SoH_percent"], res["SoH_std_estimate"],
                    res["cycles_to_target"], res["cycles_to_lower"],
                    res["cycles_per_percent_used"], False,
                    cfg.MODEL_DIR/f"{fp.stem}_projection.png")
    with (cfg.MODEL_DIR/f"{fp.stem}_prediction.json").open("w") as f: json.dump(res, f, indent=2)
    print(json.dumps(res, indent=2))

# =========================
# 4. RUN (works in notebook)
# =========================
main([])          # <- no CLI flags, ignores Jupyter’s hidden ‘-f …’


ModuleNotFoundError: No module named 'eis_helpers'